**find_centroid:**

Purpose: Calculates the centroid (or center of mass) of black pixels in a specified region of a binary image.
Parameters:
image: A binary (black and white) PIL image object.
left, right, top, bottom: Integer boundaries defining the rectangular subregion within the image.
Returns: A tuple (cx, cy) with the x and y coordinates of the centroid of black pixels in the specified region. If no black pixels are found, it returns the midpoint of the region.
Behavior: Iterates through the pixels in the specified region to calculate the average x and y coordinates of black pixels.

**find_transitions:**

Purpose: Counts the number of transitions from black (0) to white (255) pixels in a specified rectangular region of the image.
Parameters:
image: A binary (black and white) PIL image object.
left, right, top, bottom: Integer boundaries defining the subregion within the image.
Returns: An integer representing the count of black-to-white transitions within the specified region.
Behavior: Iterates over each pixel in the region and tracks transitions by comparing each pixel with the previous one.

**draw_centroid_grid:**

Purpose: Recursively divides a specified region of the image into smaller cells, computes features (centroid, transitions, and aspect ratio) at each depth level, and records them in provided lists.
Parameters:
image: A binary (black and white) PIL image object.
draw: A PIL.ImageDraw object used for drawing lines on the image to create a grid.
left, right, top, bottom: Integer boundaries defining the region within the image to divide.
depth: Integer representing the current level of recursive division (default is 0).
centroids, transitions, ratios: Lists to collect the calculated centroids, transition counts, and aspect ratios for each cell at the final depth level.
Behavior: Recursively divides the region into a 4x4 grid (64 cells) by splitting until a depth of 3 is reached. At the final depth, it calculates and stores the centroid, transition count, and aspect ratio for each cell.


**process_signature:**

Purpose: Processes a single signature image, extracts features (centroids, transitions, aspect ratios) across a 4x4 grid, and stores them in specified lists.
Parameters:
image_path: String path to the signature image file to be processed.
centroids, transitions, ratios: Lists for collecting centroids, transition counts, and aspect ratios for each cell in the 4x4 grid.
Behavior: Converts the image to grayscale, applies a binary threshold, divides it into a grid, and uses draw_centroid_grid to extract features for each cell, appending them to the given lists.
process_all_signatures:

Purpose: Processes multiple signature images in sequence, extracting features from each and saving them in a single CSV file.
Parameters:
output_prefix: String prefix for the output file, including directory path if needed.
Behavior: Iterates over 25 signature image files named R001.png to R025.png. For each, it calls process_signature to calculate and store the centroid, transition count, and aspect ratio for each cell in the 4x4 grid. Saves the collected data in a single CSV file with columns for each feature.
Returns: The path of the saved CSV file containing the extracted features.

#Preprocessing

In [28]:
from PIL import Image

# Load the image
img = Image.open('/content/R001.png')

# Convert to grayscale
gray_img = img.convert('L')

# Apply a binary threshold to convert to black and white
# Threshold value: 128, pixel values below will be black (0), above will be white (255)
threshold = 128
binary_img = gray_img.point(lambda p: p > threshold and 255)

# Save or display the result
binary_img.show()  # To display in your local environment, or you can save it
binary_img.save('binary_signature_R001.png')


#Tasks

##Task 1: Developing a bounding box

In [29]:
from PIL import Image, ImageDraw
import numpy as np
import os
import csv
import cv2
from sklearn.decomposition import PCA

In [30]:
def find_signature_bounding_box(image_path, min_signature_area=50):
    # Load the image
    image = Image.open(image_path)

    # Get the dimensions of the image
    width, height = image.size

    # Initialise bounding box coordinates
    left = width
    right = 0
    top = height
    bottom = 0

    # Count the number of black pixels (signature part) within the bounding box
    black_pixel_count = 0

    # Iterate through the image pixel by pixel
    for x in range(width):
        for y in range(height):
            color = image.getpixel((x, y))

            # Check if the pixel is black (signature part)
            if color == 0:  # 0 means black in binary image mode
                black_pixel_count += 1
                if x > right:
                    right = x
                if x < left:
                    left = x
                if y > bottom:
                    bottom = y
                if y < top:
                    top = y

    # If the number of black pixels is smaller than min_signature_area, ignore
    if black_pixel_count < min_signature_area:
        print("Signature is too small, ignoring.")
        return None

    # Return the bounding box coordinates
    return (left, top, right, bottom)

In [32]:

image_path = '/content/binary_signature_R001.png'
bounding_box = find_signature_bounding_box(image_path, min_signature_area=50)

if bounding_box:
    print(f"Bounding Box: {bounding_box}")

    # To visualize the bounding box on the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    draw.rectangle(bounding_box, outline='red', width=8)

    # Save the image with the bounding box
    output_image_path = '/content/signed_image_R001_with_box.jpg'
    image.save(output_image_path)

    # Display the saved image
    image.show()
else:
    print("No valid signature found.")


Bounding Box: (4, 8, 2870, 706)


##Task 2: Locating the centroid

In [33]:
def find_signature_centroid(image_path, bounding_box):
    # Load the image
    image = Image.open(image_path)

    # Get bounding box coordinates
    left, top, right, bottom = bounding_box

    # Initialise centroid coordinates and counter
    cx = 0
    cy = 0
    n = 0  # Number of black pixels (signature pixels) in the bounding box

    # Iterate through the bounding box pixels
    for x in range(left, right + 1):
        for y in range(top, bottom + 1):
            color = image.getpixel((x, y))

            # If the pixel is black (signature part)
            if color == 0:  # 0 means black in binary image mode
                cx += x
                cy += y
                n += 1

    # If there are black pixels, compute the centroid
    if n > 0:
        cx /= n
        cy /= n
    else:
        cx, cy = None, None  # No signature found

    return (cx, cy)

In [34]:
centroid = find_signature_centroid('/content/binary_signature_R001.png', bounding_box)
print(f"Centroid: {centroid}")

Centroid: (1359.2515635116104, 294.0762166022573)


##Task 3: Dividing the image at centroid to create four segments

In [36]:
def divide_image_at_centroid(image_path, bounding_box, centroid):
    # Load the image
    image = Image.open(image_path)

    # Get bounding box coordinates
    left, top, right, bottom = bounding_box
    cx, cy = centroid

    # Define the four segments based on the centroid
    top_left = image.crop((left, top, cx, cy))  # Segment 1: (left, cx, top, cy)
    top_right = image.crop((cx, top, right, cy))  # Segment 2: (cx, right, top, cy)
    bottom_left = image.crop((left, cy, cx, bottom))  # Segment 3: (left, cx, cy, bottom)
    bottom_right = image.crop((cx, cy, right, bottom))  # Segment 4: (cx, right, cy, bottom)

    return top_left, top_right, bottom_left, bottom_right

def draw_lines_and_save(image_path, bounding_box, centroid, output_path='signature_with_4_segments.png'):

    # Load the image
    img = Image.open(image_path)

    # Get image dimensions
    width, height = img.size

    # Draw bounding box and centroid lines
    draw = ImageDraw.Draw(img)

    # Draw the bounding box
    left, top, right, bottom = bounding_box
    draw.rectangle([left, top, right, bottom], outline="red", width=2)

    # Draw the dividing lines through the centroid
    cx, cy = centroid
    draw.line([(cx, top), (cx, bottom)], fill="green", width=5)  # Vertical line at cx
    draw.line([(left, cy), (right, cy)], fill="green", width=5)  # Horizontal line at cy

    # Save the result image
    img.save(output_path)

    # Return the saved image path
    return output_path


In [37]:
image_path = '/content/signed_image_R001_with_box.jpg'
centroid = (centroid[0], centroid[1])  # Example centroid coordinates (cx, cy)

# Divide the image into four segments
top_left, top_right, bottom_left, bottom_right = divide_image_at_centroid(image_path, bounding_box, centroid)

# Show or save the segments
top_left.show()
top_right.show()
bottom_left.show()
bottom_right.show()

# Draw centroid lines on the original image and save it
output_image_path = 'signature_R001_with_segments.png'
saved_path = draw_lines_and_save(image_path, bounding_box, centroid, output_path=output_image_path)

print(f"Saved Image Path: {saved_path}")

Saved Image Path: signature_R001_with_segments.png


##Task 4: Finding black to white transitions

In [40]:
from PIL import Image

def count_black_to_white_transitions(image, left, top, right, bottom):
    transitions = 0
    prev_pixel = image.getpixel((left, top))  # Initialize with the first pixel in the segment

    # Iterate through each pixel in the given segment
    for x in range(left + 1, right + 1):
        for y in range(top + 1, bottom + 1):
            curr_pixel = image.getpixel((x, y))
            # Check if there is a black-to-white transition
            if curr_pixel == 255 and prev_pixel == 0:
                transitions += 1
            prev_pixel = curr_pixel  # Update previous pixel

    return transitions



In [41]:
# Load the binary image
image_path = '/content/binary_signature_R001.png'
image = Image.open(image_path).convert('L')  # Convert to grayscale
threshold = 128
binary_image = image.point(lambda p: p > threshold and 255)  # Binarize the image

# Define bounding box and centroid (assuming you already have these)
bounding_box = (50, 100, 300, 350)  # Example bounding box coordinates
centroid = (175, 225)  # Example centroid coordinates

# Divide the image into four segments based on the bounding box and centroid
left, top, right, bottom = bounding_box
cx, cy = centroid

# Define the segments
top_left = (left, top, cx, cy)
top_right = (cx, top, right, cy)
bottom_left = (left, cy, cx, bottom)
bottom_right = (cx, cy, right, bottom)

# Count black-to-white transitions for each segment
transitions_top_left = count_black_to_white_transitions(binary_image, *top_left)
transitions_top_right = count_black_to_white_transitions(binary_image, *top_right)
transitions_bottom_left = count_black_to_white_transitions(binary_image, *bottom_left)
transitions_bottom_right = count_black_to_white_transitions(binary_image, *bottom_right)

print(f"Top-left transitions: {transitions_top_left}")
print(f"Top-right transitions: {transitions_top_right}")
print(f"Bottom-left transitions: {transitions_bottom_left}")
print(f"Bottom-right transitions: {transitions_bottom_right}")


Top-left transitions: 22
Top-right transitions: 192
Bottom-left transitions: 125
Bottom-right transitions: 181


##Task5:Dividing the signature into 64 cells

In [42]:
def find_centroid(image, left, right, top, bottom):
    """
    Finds the centroid of the black pixels within the specified bounding box.
    """
    cx, cy, n = 0, 0, 0

    for x in range(left, right):
        for y in range(top, bottom):
            color = image.getpixel((x, y))
            if color == 0:  # Black pixel
                cx += x
                cy += y
                n += 1

    if n > 0:
        return (cx // n, cy // n)  # Return the integer coordinates of the centroid
    else:
        # If no black pixels are found, return the geometric center
        return ((left + right) // 2, (top + bottom) // 2)

In [43]:
def draw_centroid_grid(image, draw, left, right, top, bottom, depth=0):
    """
    Recursively divide the image into 64 cells using the centroid of each cell
    and draw lines to visualize the grid.
    """
    if depth < 3:
        # Find the centroid for the current cell
        cx, cy = find_centroid(image, left, right, top, bottom)

        # Draw the dividing lines based on the centroid
        draw.line([(cx, top), (cx, bottom)], fill="blue", width=1)  # Vertical line at cx
        draw.line([(left, cy), (right, cy)], fill="blue", width=1)  # Horizontal line at cy

        # Recursively split the four quadrants based on the centroid
        draw_centroid_grid(image, draw, left, cx, top, cy, depth + 1)  # Top-left
        draw_centroid_grid(image, draw, cx, right, top, cy, depth + 1)  # Top-right
        draw_centroid_grid(image, draw, left, cx, cy, bottom, depth + 1)  # Bottom-left
        draw_centroid_grid(image, draw, cx, right, cy, bottom, depth + 1)  # Bottom-right

In [44]:
def process_signature_with_centroid_grid(image_path, output_path):
    """
    Process the image, divide it into 64 centroid-based cells, and draw grid lines for visualization.
    """
    # Load the image
    img = Image.open(image_path)

    # Convert the image to grayscale and then binary (black and white)
    img = img.convert('L')
    threshold = 128  # Set the threshold for binarization
    binary_img = img.point(lambda p: p > threshold and 255)

    # Prepare to draw lines on the original image
    draw = ImageDraw.Draw(binary_img)

    # Get image dimensions
    width, height = binary_img.size

    # Recursively divide the image into 64 cells and draw grid lines
    draw_centroid_grid(binary_img, draw, 0, width, 0, height, depth=0)

    # Save the image with the grid
    binary_img.save(output_path)
    binary_img.show()


In [47]:
process_signature_with_centroid_grid('/content/binary_signature_R001.png', '/content/signature_R001_with_centroid_64_cells.png')
print("Image with 64 segments saved successfully")

Image with 64 segments saved successfully


##Task6: Finding Centroids , Transitions  ratios for R001.png into seperate txt files

In [51]:
def find_centroid(image, left, right, top, bottom):
    cx, cy, n = 0, 0, 0

    for x in range(left, right):
        for y in range(top, bottom):
            color = image.getpixel((x, y))
            if color == 0:  # Black pixel
                cx += x
                cy += y
                n += 1

    if n > 0:
        return (cx // n, cy // n)
    else:
        return ((left + right) // 2, (top + bottom) // 2)

def find_transitions(image, left, right, top, bottom):
    transitions = 0
    prev_pixel = None

    for x in range(left, right):
        for y in range(top, bottom):
            curr_pixel = image.getpixel((x, y))
            if prev_pixel == 0 and curr_pixel == 255:
                transitions += 1
            prev_pixel = curr_pixel
    return transitions

Features extraced successfully


##Extracting Feature from one image R001

In [52]:
def draw_centroid_grid(image, draw, left, right, top, bottom, depth=0, centroids=[], transitions=[], ratios=[]):
    if depth < 3:
        cx, cy = find_centroid(image, left, right, top, bottom)

        draw.line([(cx, top), (cx, bottom)], fill="blue", width=1)
        draw.line([(left, cy), (right, cy)], fill="blue", width=1)

        draw_centroid_grid(image, draw, left, cx, top, cy, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, cx, right, top, cy, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, left, cx, cy, bottom, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, cx, right, cy, bottom, depth + 1, centroids, transitions, ratios)
    else:
        # When depth 3 is reached, calculate and store the features for each cell
        centroid = find_centroid(image, left, right, top, bottom)
        centroids.append(centroid)

        transition_count = find_transitions(image, left, right, top, bottom)
        transitions.append(transition_count)

        aspect_ratio = (right - left) / (bottom - top)
        ratios.append(aspect_ratio)


In [53]:
def process_signature(image_path, output_prefix):
    img = Image.open(image_path)
    img = img.convert('L')
    threshold = 128
    binary_img = img.point(lambda p: p > threshold and 255)

    draw = ImageDraw.Draw(binary_img)
    width, height = binary_img.size

    centroids, transitions, ratios = [], [], []

    draw_centroid_grid(binary_img, draw, 0, width, 0, height, depth=0, centroids=centroids, transitions=transitions, ratios=ratios)

    # Save the extracted features to text files
    np.savetxt(f'{output_prefix}_centroids.txt', centroids, fmt='%.2f')
    np.savetxt(f'{output_prefix}_transitions.txt', transitions, fmt='%d')
    np.savetxt(f'{output_prefix}_ratios.txt', ratios, fmt='%.2f')

    binary_img.save(f'{output_prefix}_signature_with_grid.png')
    binary_img.show()

In [54]:

process_signature('/content/binary_signature_R001.png', '/content/latestresult_R001')
print("Features extraced successfully")

Features extraced successfully


In [63]:
def find_centroid(image, left, right, top, bottom):
    cx, cy, n = 0, 0, 0

    for x in range(left, right):
        for y in range(top, bottom):
            color = image.getpixel((x, y))
            if color == 0:  # Black pixel
                cx += x
                cy += y
                n += 1

    if n > 0:
        return (cx // n, cy // n)
    else:
        return ((left + right) // 2, (top + bottom) // 2)



In [ ]:
def find_transitions(image, left, right, top, bottom):
    transitions = 0
    prev_pixel = None

    for x in range(left, right):
        for y in range(top, bottom):
            curr_pixel = image.getpixel((x, y))
            if prev_pixel == 0 and curr_pixel == 255:
                transitions += 1
            prev_pixel = curr_pixel
    return transitions

In [ ]:
def draw_centroid_grid(image, draw, left, right, top, bottom, depth=0, centroids=[], transitions=[], ratios=[]):
    if depth < 3:
        cx, cy = find_centroid(image, left, right, top, bottom)

        draw.line([(cx, top), (cx, bottom)], fill="blue", width=1)
        draw.line([(left, cy), (right, cy)], fill="blue", width=1)

        draw_centroid_grid(image, draw, left, cx, top, cy, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, cx, right, top, cy, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, left, cx, cy, bottom, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, cx, right, cy, bottom, depth + 1, centroids, transitions, ratios)
    else:
        # When depth 3 is reached, calculate and store the features for each cell
        centroid = find_centroid(image, left, right, top, bottom)
        centroids.append(centroid)

        transition_count = find_transitions(image, left, right, top, bottom)
        transitions.append(transition_count)

        aspect_ratio = (right - left) / (bottom - top)
        ratios.append(aspect_ratio)

In [ ]:
def process_signature(image_path, output_prefix):
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_prefix)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    img = Image.open(image_path)
    img = img.convert('L')
    threshold = 128
    binary_img = img.point(lambda p: p > threshold and 255)

    draw = ImageDraw.Draw(binary_img)
    width, height = binary_img.size

    centroids, transitions, ratios = [], [], []

    draw_centroid_grid(binary_img, draw, 0, width, 0, height, depth=0, centroids=centroids, transitions=transitions, ratios=ratios)

    # Prepare CSV file data
    csv_data = []
    for i in range(64):
        centroid_x, centroid_y = centroids[i]
        transition_count = transitions[i]
        aspect_ratio = ratios[i]
        csv_data.append([centroid_x, centroid_y, transition_count, aspect_ratio])

    # Save the data into a CSV file
    csv_file = f'{output_prefix}_features.csv'
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Centroid X', 'Centroid Y', 'Transitions', 'Aspect Ratio'])  # Header
        writer.writerows(csv_data)

    # Save the image with the grid
    grid_image_path = f'{output_prefix}_signature_with_grid_R001.png'
    binary_img.save(grid_image_path)
    binary_img.show()

    return csv_file, grid_image_path

In [64]:
csv_file, grid_image = process_signature('/content/binary_signature_R001.png', '/content/latestresult_of_R001')
print(f'CSV File Created: {csv_file}')
print(f'Grid Image Saved: {grid_image}')

CSV File Created: /content/latestresult_of_R001_features.csv
Grid Image Saved: /content/latestresult_of_R001_signature_with_grid_R001.png


##Extracting the features of all 25 Images into result.csv

In [55]:
def find_centroid(image, left, right, top, bottom):
    cx, cy, n = 0, 0, 0
    for x in range(left, right):
        for y in range(top, bottom):
            color = image.getpixel((x, y))
            if color == 0:  # Black pixel
                cx += x
                cy += y
                n += 1
    if n > 0:
        return (cx // n, cy // n)
    else:
        return ((left + right) // 2, (top + bottom) // 2)




In [56]:
def find_transitions(image, left, right, top, bottom):
    transitions = 0
    prev_pixel = None
    for x in range(left, right):
        for y in range(top, bottom):
            curr_pixel = image.getpixel((x, y))
            if prev_pixel == 0 and curr_pixel == 255:
                transitions += 1
            prev_pixel = curr_pixel
    return transitions

In [57]:
def draw_centroid_grid(image, draw, left, right, top, bottom, depth=0, centroids=[], transitions=[], ratios=[]):
    if depth < 3:
        cx, cy = find_centroid(image, left, right, top, bottom)
        draw.line([(cx, top), (cx, bottom)], fill="blue", width=1)
        draw.line([(left, cy), (right, cy)], fill="blue", width=1)
        draw_centroid_grid(image, draw, left, cx, top, cy, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, cx, right, top, cy, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, left, cx, cy, bottom, depth + 1, centroids, transitions, ratios)
        draw_centroid_grid(image, draw, cx, right, cy, bottom, depth + 1, centroids, transitions, ratios)
    else:
        centroid = find_centroid(image, left, right, top, bottom)
        centroids.append(centroid)
        transition_count = find_transitions(image, left, right, top, bottom)
        transitions.append(transition_count)
        aspect_ratio = (right - left) / (bottom - top)
        ratios.append(aspect_ratio)

In [58]:
def process_signature(image_path, centroids, transitions, ratios):
    img = Image.open(image_path)
    img = img.convert('L')
    threshold = 128
    binary_img = img.point(lambda p: p > threshold and 255)
    draw = ImageDraw.Draw(binary_img)
    width, height = binary_img.size
    draw_centroid_grid(binary_img, draw, 0, width, 0, height, depth=0, centroids=centroids, transitions=transitions, ratios=ratios)

In [59]:
def process_all_signatures(output_prefix):
    centroids, transitions, ratios = [], [], []
    csv_file = f'{output_prefix}_features.csv'

    # Process each file from R001.png to R025.png
    for i in range(1, 26):
        file_path = f'/content/R{i:03d}.png'
        process_signature(file_path, centroids, transitions, ratios)

    # Prepare CSV file data
    csv_data = []
    for i in range(len(centroids)):
        centroid_x, centroid_y = centroids[i]
        transition_count = transitions[i]
        aspect_ratio = ratios[i]
        csv_data.append([centroid_x, centroid_y, transition_count, aspect_ratio])

    # Write the results to a single CSV file
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Centroid X', 'Centroid Y', 'Transitions', 'Aspect Ratio'])  # Header
        writer.writerows(csv_data)

    print(f'All data saved to CSV: {csv_file}')
    return csv_file

In [60]:
csv_file = process_all_signatures('/content/Output')
print(f'CSV File Created: {csv_file}')


All data saved to CSV: /content/Output_features.csv
CSV File Created: /content/Output_features.csv


#Finding Skew and Slant Angles

In [67]:
def calculate_skew_angle(image_path):
    # Load image, convert to grayscale and binary
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)

    # Detect skew angle using PCA
    coords = np.column_stack(np.where(binary_img > 0))
    pca = PCA(n_components=2)
    pca.fit(coords)

    angle = np.arctan2(pca.components_[0, 1], pca.components_[0, 0])
    skew_angle = np.degrees(angle)

    return skew_angle

In [68]:
def calculate_slant_angle(image_path):
    # Load image, convert to grayscale and binary
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_img = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)

    # Calculate the slant angle using projection profile
    height, width = binary_img.shape
    column_sums = []

    for x in range(width):
        column = binary_img[:, x]
        y_coords = np.where(column > 0)[0]
        if y_coords.size > 0:
            column_sums.append(np.mean(y_coords))

    # Fit a line to the center points
    x_coords = np.arange(len(column_sums))
    if len(x_coords) > 1:
        fit = np.polyfit(x_coords, column_sums, 1)
        slant_angle = np.degrees(np.arctan(fit[0]))
    else:
        slant_angle = 0  # Default if no angle can be calculated

    return slant_angle

In [69]:
image_path = '/content/R001.png'
skew_angle = calculate_skew_angle(image_path)
slant_angle = calculate_slant_angle(image_path)

print(f"Skew Angle: {skew_angle:.2f}°")
print(f"Slant Angle: {slant_angle:.2f}°")

Skew Angle: 92.90°
Slant Angle: -2.57°


Each function is designed to support feature extraction from signature images by analyzing structural properties at a grid-cell level. The code recursively divides images into smaller regions, calculates centroids, counts transitions, and finds aspect ratios, all of which are saved in a structured CSV format for easy analysis. This design allows for flexible processing of multiple images by storing all relevant features in a single output file.